In [1]:
import scvelo as scv
import scanpy as sc
import pandas as pd
import numpy as np
import os
import loompy

# scVelo

In [2]:
# Load rawcounts already processed
adata = sc.read("../data/data_RNAvelocity/processed_counts.txt",sep=' ')

# Since python like cells as rows -> transpose matrix
adata = adata.T

# Insert umap coordinates from SO
umap_cord = pd.read_csv("../data/data_RNAvelocity/coordinates_umap_scvelo.txt", sep=",") 
adata.obsm["X_umap"] = umap_cord.iloc[:,0:2].values

# Read metadata
metadata = pd.read_csv("../data/data_RNAvelocity/metadata_GEO.txt", sep= ",")

In [3]:
# Add metadata file (OBS)
metadata = pd.DataFrame(metadata)
adata.obs["batch_group"] = metadata["batch_group"].values
adata.obs["cluster"] = umap_cord.iloc[:,2].values

# Add features (VAR)
feat = pd.read_csv("../data/data_RNAvelocity/processed_counts.txt", sep= " ")
feat = feat.index
feat = pd.DataFrame(feat)
feat = pd.DataFrame({"feature" : feat.iloc[:,0]})
adata.var["features"] = feat.values

In [4]:
                                    # Read LOOM file
ldata = scv.read("../data/data_RNAvelocity/s_un_am_allgenes.loom", cache=True)
# Merge adata with loom file
adata = scv.utils.merge(adata, ldata)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [5]:
#we compute the first- and second-order moments (basically means and variances) for velocity estimation:
scv.pp.moments(adata)

Normalized count data: spliced, unspliced.
computing neighbors
    finished (0:00:05) --> added to `.uns['neighbors']`
    'distances', weighted adjacency matrix
    'connectivities', weighted adjacency matrix
computing moments based on connectivities
    finished (0:00:02) --> added 
    'Ms' and 'Mu', moments of spliced/unspliced abundances (adata.layers)


In [6]:
                                        # Estimates of velocity
#For steady state model
#scv.tl.velocity(adata, mode='steady_state')

#For dynamic model run 
scv.tl.recover_dynamics(adata)
scv.tl.velocity(adata, mode='dynamical')

#The velocities are stored in adata.layers just like the count matrices.
scv.tl.velocity_graph(adata)

recovering dynamics
    finished (0:11:26) --> added 
    'fit_pars', fitted parameters for splicing dynamics (adata.var)
computing velocities
    finished (0:00:04) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)
computing velocity graph
    finished (0:00:03) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)


In [ ]:
# Computes confidence of velocities
scv.tl.velocity_confidence(adata)

scv.pl.scatter(adata, color='velocity_length', perc=[10,98], size= 100)
scv.pl.scatter(adata, color='velocity_confidence', perc=[2,98], size= 100)

In [7]:
# Extract highly variable genes
scv.pp.filter_genes_dispersion(adata)
HVG = adata.var[adata.var["velocity_genes"] == True].index

In [8]:
len(HVG)

1337

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap', color = "cluster",                                 
                                 size = 80.0, alpha = 0.8,
                                 palette= ['#A1DAB4','#FEE391','#41B6C4','#1D91C0'], legend_loc = "none")

In [ ]:
?scv.tl.terminal_states

In [ ]:
# Compute pseudotime
scv.tl.terminal_states(adata)
scv.tl.velocity_pseudotime(adata)

scv.pl.scatter(adata, color='velocity_pseudotime', color_map='bwr')

In [ ]:
# graoh with root and end cells

scv.pl.scatter(adata, color=['root_cells', 'end_points'])

In [ ]:
scv.logging.print_versions()

# Plot expression of a gene

In [ ]:
# Visualization
#scv.pl.velocity(adata, var_names=['Stmn1'])
#scv.pl.velocity_graph(adata)